# MCNN GTSRB Dataset

## Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
! unzip gtsrb-german-traffic-sign.zip

In [ ]:
pip install torchmetrics

## GTSRB Complete EDA

In [ ]:
import os
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, RandomSampler, random_split
from torchvision import transforms, utils, datasets
from google.colab import files
from skimage import io, transform, color
from trafficsignsdataset import GTSRBDataset
from numpy.ma.core import MaskedConstant
import torch.utils.data.dataloader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
from sklearn import metrics, svm
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from morph2d import Morph2d
from pseudoinversemod import pseudoInverse
from tabulate import tabulate
import datetime
import scipy
from torchmetrics.functional import roc
from CIDeLong import delong_roc_variance
from scipy import stats
from MCNN import MCNN

In [ ]:
"GTSRB EDA"

def eda_gtsrb(train_path, test_path):
    # Read DataFrames
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Number of images
    print('The train set contains {} images, and the test set contains {} images.\
 We have a total of {} images'.format(len(train_df),
                                      len(test_df),
                                      len(train_df) + len(test_df)))
    # Size of images
    complete_w_df = pd.concat([train_df["Width"], test_df["Width"]], axis = 1)
    complete_w_df.columns = ["Train", "Test"]

    plt.figure(1)
    complete_w_df.plot.hist(figsize=(10,6), color = ["Gray", "Blue"])
    plt.title("Width of images in GTSRB dataset", fontweight="bold")
    plt.xlabel("Pixels")
    plt.ylabel("No. of images")
    plt.show
    #plt.savefig('/content/WidthHistCompleteDataset.png')

    # Number of classes
    complete_c_df = pd.concat([train_df["ClassId"], test_df["ClassId"]], axis = 1)
    complete_c_df.columns = ["Train", "Test"]

    plt.figure(2)
    complete_c_df.plot.hist(bins = 43, figsize=(10,6), color = ["Gray", "Blue"])
    plt.title("Classes in GTSRB dataset", fontweight="bold")
    plt.xlabel("Class")
    plt.ylabel("No. of images")
    plt.show
    #plt.savefig('/content/ClassesHistCompleteDataset.png')

    # Chosen classes
    print("The classes chosen were: class 1 with {} training images and {} testing images,\
class 13 with {} training images and {} testing images".format(len(train_df[train_df["ClassId"]==1]),
                                                 len(test_df[test_df["ClassId"]==1]),
                                                 len(train_df[train_df["ClassId"]==13]),
                                                 len(test_df[test_df["ClassId"]==13])))
    
    # Size of images after filter by class
    train_class1 = train_df[train_df["ClassId"]==1]
    train_class2 = train_df[train_df["ClassId"]==2]
    test_class1 = test_df[test_df["ClassId"]==1]
    test_class2 = test_df[test_df["ClassId"]==2]

    filtered_c_df = pd.concat([train_class1["Width"],
                               train_class2["Width"],
                               test_class1["Width"],
                               test_class2["Width"]], axis = 1)
    
    filtered_c_df.columns = ["Train Class 1",
                             "Train Class 2",
                             "Test Class 1",
                             "Test Class 2"]

    plt.figure(3)
    filtered_c_df.plot.hist(figsize=(10,6), color = ["Black", "Gray", "Purple", "Blue"])
    plt.title("Width of images in GTSRB dataset for 2 classes", fontweight="bold")
    plt.xlabel("Pixels")
    plt.ylabel("No. of images")
    plt.show
    #plt.savefig('/content/WidthHist2ClassesDataset.png')


train_path = '/content/Train.csv'
test_path = '/content/Test.csv'

eda_gtsrb(train_path, test_path)

## Data Preparation

In [ ]:
def FilterData(train_csv_file, test_csv_file, root_dir):
    """This function filters the GTSRB dataset. It takes images with a size
    lower than 90 pixels and classes 1 and 13"""

    # Variables definition
    train_gtsrb = pd.read_csv(train_csv_file).sample(frac=1)
    test_gtsrb = pd.read_csv(test_csv_file).sample(frac=1)
    root_dir = root_dir
    root_fname = '/content/FilteredData/'

    # Folders creation
    os.mkdir("/content/FilteredData")
    os.mkdir("/content/FilteredData/Train")
    os.mkdir("/content/FilteredData/Test")
    os.mkdir("/content/FilteredData/Train/1")
    os.mkdir("/content/FilteredData/Train/13")

    train_gtsrb = train_gtsrb.sample(frac=1)
    test_gtsrb = test_gtsrb.sample(frac=1)
    gtsrb = pd.concat([train_gtsrb, test_gtsrb], axis = 0)

    train_gtsrb_new = pd.DataFrame(gtsrb.iloc[0,:], columns = train_gtsrb.columns)
    test_gtsrb_new = pd.DataFrame(gtsrb.iloc[0,:], columns = test_gtsrb.columns)

    # Filter data
    for idx in range(len(train_gtsrb)):
        if train_gtsrb.iloc[idx,0]<150:
            if train_gtsrb.iloc[idx,6]==1 or train_gtsrb.iloc[idx,6]==13:
                img_name = os.path.join(root_dir,
                                    train_gtsrb.iloc[idx, 7])
                image = io.imread(img_name)
                train_gtsrb_new = train_gtsrb_new.append(train_gtsrb.iloc[idx,:],
                                                         ignore_index=True)

                fname = os.path.join(root_fname,
                                    train_gtsrb.iloc[idx, 7])
                io.imsave(fname, image)
                train_gtsrb_new.to_csv('/content/FilteredData/train_gtsrb.csv')

    for idx in range(len(test_gtsrb)):
        if test_gtsrb.iloc[idx,0]<150: # 90
            if test_gtsrb.iloc[idx,6]==1 or test_gtsrb.iloc[idx,6]==13:
                img_name = os.path.join(root_dir,
                                    test_gtsrb.iloc[idx, 7])
                image = io.imread(img_name)
                test_gtsrb_new = test_gtsrb_new.append(test_gtsrb.iloc[idx,:],
                                                       ignore_index=True)

                fname = os.path.join(root_fname,
                                    test_gtsrb.iloc[idx, 7])
                io.imsave(fname, image)
                test_gtsrb_new.to_csv('/content/FilteredData/test_gtsrb.csv')


In [ ]:
FilterData(train_csv_file='/content/Train.csv', test_csv_file='/content/Test.csv', root_dir='/content/')

## Main Code

In [ ]:
# General Settings
batch_size = 4037
test_batch_size = 1730
num_train_samples = 4037
num_test_samples = 1730
stop_train = num_train_samples/batch_size
stop_test = num_test_samples/test_batch_size
seed = 80
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

# Training settings
traffic_train_dataset = GTSRBDataset(csv_file='/content/FilteredData/train_gtsrb.csv',
                              root_dir='/content/FilteredData',
                              transform=transforms.Compose([
                                               Rescale(100),
                                               ToTensor()
                                           ]))

train_loader = torch.utils.data.DataLoader(traffic_train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           worker_init_fn=seed_worker,
                                           generator=g)

# Testing settings
traffic_test_dataset = GTSRBDataset(csv_file='/content/FilteredData/test_gtsrb.csv',
                              root_dir='/content/FilteredData',
                              transform=transforms.Compose([
                                               Rescale(100),
                                               ToTensor()
                                           ]))

test_loader = torch.utils.data.DataLoader(traffic_test_dataset,
                                          batch_size=test_batch_size,
                                          shuffle=True,
                                          worker_init_fn=seed_worker,
                                          generator=g)

# Three definitions, each per channel, for weights' independance
model_1 = MCNN()
model_2 = MCNN()
model_3 = MCNN()

optimizer_1 = pseudoInverse(params=model_1.parameters(), C=1e-3)
optimizer_2 = pseudoInverse(params=model_2.parameters(), C=1e-3)
optimizer_3 = pseudoInverse(params=model_3.parameters(), C=1e-3)

def plot_auc(fpr, tpr, auc_result):

    plt.figure(figsize=(12,7))
    plt.plot(fpr, tpr, linewidth=1, color='blue')
    plt.plot(np.linspace(0, max(fpr), len(fpr)),
             np.linspace(0, max(tpr), len(tpr)),
             "--", linewidth=0.8, color="black")
    plt.title(label="ROC-AUC. MCNN",
             fontsize=15,
             fontweight="bold")
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    plt.legend(["AUC: " + str(round(auc_result.item(), 3))], loc='right')
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.xlim(right=max(tpr))
    plt.ylim(top=max(fpr))

def final_classifier(out_train_1, out_train_2, out_train_3, out_test_1,
                     out_test_2, out_test_3, target_train, target_test):
    """This function performs a final classification considering the independent
    DPFs obtained from the MCNN method.
    
    inputs:
      out_trains = DPFs obtained per channel from the training set, it contains
                   the probabilities of both classes [# classes, # subjects].
      out_tests = DPFs obtained per channel from the testing set, it contains
                  the probabilities of both classes [# classes, # subjects].
      target_train = Labels corresponding to the subjects in the training set.
      target_test = Labels corresponding to the subjects in the testing set."""

    np.random.seed(seed)
    
    # Initial definitions
    model1 = GaussianNB()
    model2 = svm.SVC(random_state=seed, probability=True)
    model3 = AdaBoostClassifier(random_state=seed)
    model4 = RandomForestClassifier(random_state=seed)
    model = [model1, model2, model3, model4]

    acc_train = []
    acc_test = []
    acc_balanced = []
    ci_bacc = []
    ci_acc = []
    conf_mat = []
    auc = []
    auc_delong = []
    auc_cov = []
    ci_auc = []
    err = []
    fpr_all = []
    tpr_all = []
    p_fpr_all = []
    p_tpr_all = []

    count = 1
    alpha = 0.95
    confidence = 0.95
    z_value = stats.norm.ppf((1 + confidence) / 2.0)

    acc_train.append("Train set accuracy")
    acc_test.append("Test set accuracy")
    acc_balanced.append("Balanced accuracy")
    ci_bacc.append("CI Balanced acc")
    ci_acc.append("CI Acc")
    auc.append("ROC AUC score")
    auc_delong.append("ROC AUC DeLong")
    ci_auc.append("CI AUC")
    auc_cov.append("AUC COV")
    err.append("Error")
    headers = ["Metrics", "Gaussian Naive Bayes", "SVM",
                        "AdaBoost", "Random Forest"]

    # Input data preparation
    out_train_1 = out_train_1.detach().numpy()
    out_train_2 = out_train_2.detach().numpy()
    out_train_3 = out_train_3.detach().numpy()

    joint_train = np.vstack((out_train_1[:,0],
                             out_train_2[:,0],
                             out_train_3[:,0]))
    joint_train = np.transpose(joint_train)
    joint_train = pd.DataFrame(joint_train)

    out_test_1 = out_test_1.detach().numpy()
    out_test_2 = out_test_2.detach().numpy()
    out_test_3 = out_test_3.detach().numpy()

    joint_test = np.vstack((out_test_1[:,0],
                            out_test_2[:,0],
                            out_test_3[:,0]))
    joint_test = np.transpose(joint_test)
    joint_test = pd.DataFrame(joint_test)

    # Training
    for mod in model:
        trained_model = mod.fit(joint_train,target_train)
        trained_model.fit(joint_train, target_train)
        acc_train_ind = metrics.accuracy_score(target_train,
                                               trained_model.predict(joint_train))
        acc_train.append(acc_train_ind)

    # Testing
    for mod in model: 
        predicted = mod.predict(joint_test)
        probs = mod.predict_proba(joint_test)
        acc_test_ind = metrics.accuracy_score(target_test, predicted)
        auc_ind = roc_auc_score(target_test, probs[:,1])
        acc_balanced_ind = balanced_accuracy_score(target_test, predicted)
        fpr, tpr, thresholds = roc_curve(target_test, probs[:,1], pos_label=1)
        p_fpr, p_tpr, _ = roc_curve(target_test, predicted, pos_label=1)
        err_ind = 1 - acc_test_ind

        auc_ind_delong, auc_ind_cov = delong_roc_variance(target_test,
                                                          probs[:,1])
        
        # AUC CIs
        auc_std = np.sqrt(auc_ind_cov)
        lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
        ci_auc_ind = stats.norm.ppf(
            lower_upper_q,
            loc=auc_ind_delong,
            scale=auc_std)

        ci_auc_ind[ci_auc_ind > 1] = 1
        ci_auc_ind[0] = round(ci_auc_ind[0], 3)
        ci_auc_ind[1] = round(ci_auc_ind[1], 3)

        # Acc CIs
        ci_length = z_value * np.sqrt((acc_test_ind * (1 - acc_test_ind)) / joint_test.shape[0])
        ci_lower = acc_test_ind - ci_length
        ci_upper = acc_test_ind + ci_length
        ci_acc_ind = (round(ci_lower, 3), round(ci_upper, 3))

        # Balanced acc CIs
        ci_length = z_value * np.sqrt((acc_balanced_ind * (1 - acc_balanced_ind)) / joint_test.shape[0])
        ci_lower = acc_balanced_ind - ci_length
        ci_upper = acc_balanced_ind + ci_length
        ci_bacc_ind = (round(ci_lower, 3), round(ci_upper, 3))

        acc_test.append(round(acc_test_ind, 3))
        ci_acc.append(ci_acc_ind)
        acc_balanced.append(round(acc_balanced_ind, 3))
        ci_bacc.append(ci_bacc_ind)
        auc_delong.append(round(auc_ind_delong, 3))
        auc_cov.append(round(auc_ind_cov, 3))
        ci_auc.append(ci_auc_ind)
        auc.append(round(auc_ind, 3))
        err.append(round(err_ind, 3))
        fpr_all.append(fpr)
        tpr_all.append(tpr)
        p_fpr_all.append(p_fpr)
        p_tpr_all.append(p_tpr)

        conf_mat_ind = confusion_matrix(target_test, predicted)
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat_ind,
                                      display_labels=mod.classes_)
        disp.plot(cmap='Blues')
        disp.ax_.set_title(headers[count])
        count += 1
        print("\n")

        plot_auc(fpr, tpr, auc_ind)
        print("\n")

    # Print metrics resulted
    data = [acc_train, acc_test, ci_acc, acc_balanced, ci_bacc, auc_delong, ci_auc, auc_cov, err]
    print(tabulate(data, headers))
    print("\n")

    return fpr_all, tpr_all, p_fpr_all, p_tpr_all


def train():
    """This functions performs the training process"""

    print("Ensembles training process starts.........................")
    model_1.train()
    model_2.train()
    model_3.train()

    for batch_idx, sample_batched in enumerate(train_loader):
        if batch_idx > stop_train-1:
          break
        
        # Input data and targets preparation
        data = sample_batched['image'].float()
        data_shape = data.shape
        data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                              data_shape[2], data_shape[3]))
        target = sample_batched['label'].int()
        target = target.reshape([len(target)])
        data_1, data_2, data_3 = Variable(data_1), Variable(data_2), Variable(data_3)
        target = Variable(target)

        # Independent training per channel
        hiddenOut = model_1.forwardToHidden(data_1) # First channel
        optimizer_1.train(inputs=hiddenOut, targets=target)

        hiddenOut = model_2.forwardToHidden(data_2) # Second channel
        optimizer_2.train(inputs=hiddenOut, targets=target)

        hiddenOut = model_3.forwardToHidden(data_3) # Third channel
        optimizer_3.train(inputs=hiddenOut, targets=target)
    
    print("Ensembles training process ends...........................")

def train_accuracy():
    """This functions test the network using the training set.
    
       outputs:
          output_1 = PDFs from first channel [# classes, # subjects]
          output_2 = PDFs from second channel [# classes, # subjects]
          output_3 = PDFs from third channel [# classes, # subjects]
          target = labels of each subject"""

    print("Ensembles training testing starts.........................")
    model_1.eval()
    model_2.eval()
    model_3.eval()

    count_train = 0 # Temporal

    with torch.no_grad():
        for batch_idx, sample_batched in enumerate(train_loader):
            count_train = batch_idx
            if batch_idx > stop_train-1:
              break

            # Input data and targets preparation
            data = sample_batched['image'].float()
            data_shape = data.shape
            data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            target = sample_batched['label'].int()
            target = target.reshape([len(target)])
            data_1, data_2, data_3 = Variable(data_1), Variable(data_2), Variable(data_3)
            target = Variable(target)
            output_1 = model_1.forward(data_1) # First channel
            output_2 = model_2.forward(data_2) # Second channel
            output_3 = model_3.forward(data_3) # Third channel

    print("Ensembles training testing ends...........................")
    return output_1, output_2, output_3, target, count_train


def test():
    """This functions test the network using the testing set.
    
       outputs:
          output_1 = PDFs from first channel [# classes, # subjects]
          output_2 = PDFs from second channel [# classes, # subjects]
          output_3 = PDFs from third channel [# classes, # subjects]
          target = labels of each subject"""

    print("Ensembles testing starts..................................")
    model_1.eval()
    model_2.eval()
    model_3.eval()
    
    count_test = 0

    with torch.no_grad():
        for batch_idx, sample_batched in enumerate(test_loader):
            count_test = batch_idx
            if batch_idx > stop_test-1:
              break

            # Input data and targets preparation
            data = sample_batched['image'].float()
            data_shape = data.shape
            data_1 = torch.reshape(data[:,0,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_2 = torch.reshape(data[:,1,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            data_3 = torch.reshape(data[:,2,:,:],(data_shape[0],1,
                                                  data_shape[2], data_shape[3]))
            target = sample_batched['label'].int()
            target = target.reshape([len(target)])
            data_1, data_2, data_3 = Variable(data_1), Variable(data_2), Variable(data_3)
            target = Variable(target)
            output_1 = model_1.forward(data_1) # First channel
            output_2 = model_2.forward(data_2) # Second channel
            output_3 = model_3.forward(data_3) # Third channel
        
    return output_1, output_2, output_3, target, count_test

# Complete MCNN
init=time.time()
train()
train_time = time.time()
out_train_1, out_train_2, out_train_3, target_train, count_train = train_accuracy()
target_train[target_train==13]=0

out_test_1, out_test_2, out_test_3, target_test, count_test = test()
target_test[target_test==13]=0
test_time = time.time()
P_FPR, P_TPR, FPR, TPR = final_classifier(out_train_1, out_train_2, out_train_3,
                 out_test_1, out_test_2, out_test_3,
                 target_train, target_test)
ending=time.time()

print("Training time: ", str(datetime.timedelta(seconds = train_time - init)), "\n",
      "Testing time: ", str(datetime.timedelta(seconds = test_time - train_time)), "\n",
      "Final process time: ", str(datetime.timedelta(seconds = ending - test_time)), "\n",
      "Complete process time: ", str(datetime.timedelta(seconds = ending - init)))
print("\n")


In [ ]:
def ind_accuracy(output, target):
    correct = 0
    pred=output.data.max(1)[1]
    correct += pred.eq(target.data).cpu().sum()
    print('\nAccuracy: {}/{} ({:.2f}%)\n'.format(correct,
                                                 output.shape[0],
                                                 100. * correct / output.shape[0]))
    
def ind_confusion_matrix(output_aux, target_aux):

  confusion_matrix = torch.zeros([2,2])
  TP = 0
  TN = 0
  FP = 0
  FN = 0

  def binarization(output_aux):
    for indx in range(output_aux.shape[0]):
      max_val = torch.max(output_aux[indx,:])
      max_val -= 0.001
      row = F.threshold(output_aux[indx,:], max_val.item(), 0, inplace=False)
      row[row==torch.max(row).item()] = 1
      output_aux[indx,:] = row
    return output_aux

  output_aux = binarization(output_aux)

  for i in range(2):
    confusion_matrix[i,:] = torch.bincount(target_aux[output_aux[:,i].nonzero()][:,0], minlength=2)

  for i in range(2):
    TP += confusion_matrix[i,i].item()
    TN += torch.sum(confusion_matrix).item() - torch.sum(confusion_matrix[i,:]).item() - torch.sum(confusion_matrix[:,i]).item() + confusion_matrix[i,i].item()
    FP +=  torch.sum(confusion_matrix[i,:]).item() - confusion_matrix[i,i].item()
    FN +=  torch.sum(confusion_matrix[:,i]).item() - confusion_matrix[i,i].item()

  print("Confusion Matrix: \n", confusion_matrix, "\n")
  print("True Positives: ", TP)
  print("True Negatives: ", TN)
  print("False Positive: ", FP)
  print("False Negatives: ", FN)


def metrics(output_aux, target_aux):

  confusion_matrix = torch.zeros([14,14])
  TP = torch.zeros([14])
  TN = torch.zeros([14])
  FP = torch.zeros([14])
  FN = torch.zeros([14])

  def binarization(output_aux):
    for indx in range(output_aux.shape[0]):
      max_val = torch.max(output_aux[indx,:])
      max_val -= 0.001
      row = F.threshold(output_aux[indx,:], max_val.item(), 0, inplace=False)
      row[row==torch.max(row).item()] = 1
      output_aux[indx,:] = row
    return output_aux

  output_aux = binarization(output_aux)

  for i in range(14):
    confusion_matrix[i,:] = torch.bincount(target_aux[output_aux[:,i].nonzero()][:,0], minlength=14)

  for i in range(14):
    TP[i] = confusion_matrix[i,i].item()
    TN[i] = torch.sum(confusion_matrix).item() - torch.sum(confusion_matrix[i,:]).item() - torch.sum(confusion_matrix[:,i]).item() + confusion_matrix[i,i].item()
    FP[i] =  torch.sum(confusion_matrix[i,:]).item() - confusion_matrix[i,i].item()
    FN[i] =  torch.sum(confusion_matrix[:,i]).item() - confusion_matrix[i,i].item()

  mean_TP = torch.mean(TP).item()
  mean_TN = torch.mean(TN).item()
  mean_FP = torch.mean(FP).item()
  mean_FN = torch.mean(FN).item()

  print("Average True Positives: ", mean_TP)
  print("Average True Negatives: ", mean_TN)
  print("Average False Positives: ", mean_FP)
  print("Average False Negatives: ", mean_FN)


print("Red Channel \n")
print("Training:")
ind_accuracy(out_train_1, target_train)
print("Testing:")
ind_accuracy(out_test_1, target_test)
print("\n")

print("Green Channel \n")
print("Training:")
ind_accuracy(out_train_2, target_train)
print("Testing:")
ind_accuracy(out_test_2, target_test)
print("\n")

print("Blue Channel \n")
print("Training:")
ind_accuracy(out_train_3, target_train)
print("Testing:")
ind_accuracy(out_test_3, target_test)